In [ ]:
# Install Google Cloud Storage library
!pip install -q google-cloud-storage

# Authenticate your Google account
from google.colab import auth
auth.authenticate_user()


MessageError: Error: credential propagation was unsuccessful

In [ ]:
from google.cloud import storage

# Initialize a client
client = storage.Client()

# Reference the public bucket
bucket_name = 'waymo_open_dataset_end_to_end_camera_v_1_0_0'
bucket = client.bucket(bucket_name)


In [ ]:
# List all blobs (files) in the bucket
blobs = bucket.list_blobs()

for blob in blobs:
    print(blob.name)


training_202503292338.tfrecord-00000-of-00315
training_202503292338.tfrecord-00001-of-00315
training_202503292338.tfrecord-00002-of-00315
training_202503292338.tfrecord-00003-of-00315
training_202503292338.tfrecord-00004-of-00315
training_202503292338.tfrecord-00005-of-00315
training_202503292338.tfrecord-00006-of-00315
training_202503292338.tfrecord-00007-of-00315
training_202503292338.tfrecord-00008-of-00315
training_202503292338.tfrecord-00009-of-00315
training_202503292338.tfrecord-00010-of-00315
training_202503292338.tfrecord-00011-of-00315
training_202503292338.tfrecord-00012-of-00315
training_202503292338.tfrecord-00013-of-00315
training_202503292338.tfrecord-00014-of-00315
training_202503292338.tfrecord-00015-of-00315
training_202503292338.tfrecord-00016-of-00315
training_202503292338.tfrecord-00017-of-00315
training_202503292338.tfrecord-00018-of-00315
training_202503292338.tfrecord-00019-of-00315
training_202503292338.tfrecord-00020-of-00315
training_202503292338.tfrecord-000

In [ ]:
# Download a specific file
blob = bucket.blob('training_202503292338.tfrecord-00000-of-00315')
blob.download_to_filename('local_copy.tfrecord')


In [ ]:
import tensorflow as tf

tfrecord_file = "/content/local_copy.tfrecord"
# Create a dataset from the TFRecord file
raw_dataset = tf.data.TFRecordDataset(tfrecord_file)

# Preview the first few records
for raw_record in raw_dataset.take(5):
    print(repr(raw_record))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Parse the raw data to extract images
for raw_record in raw_dataset.take(1):
    # Extract raw bytes
    record_bytes = raw_record.numpy()

    # Look for JPEG signatures (0xFFD8FF..)
    jpeg_start = b'\xff\xd8\xff'
    positions = []
    pos = 0

    # Find all JPEG images in the byte stream
    while True:
        pos = record_bytes.find(jpeg_start, pos)
        if pos == -1:
            break
        positions.append(pos)
        pos += 1

    print(f"Found {len(positions)} potential JPEG images")

    # Try to decode each potential JPEG
    for i, start_pos in enumerate(positions[:8]):  # Limit to first 3 images
        # Try to find end of JPEG by looking for next header or using a large block
        end_pos = record_bytes.find(jpeg_start, start_pos + 1)
        if end_pos == -1:
            # If no next header, use a large chunk (adjust as needed)
            img_data = record_bytes[start_pos:start_pos + 1000000]
        else:
            img_data = record_bytes[start_pos:end_pos]

        try:
            # Try to decode the image
            img = tf.image.decode_jpeg(img_data)
            print(f"Successfully decoded image {i}, shape: {img.shape}")

            # Display the image
            plt.figure(figsize=(10, 8))
            plt.imshow(img.numpy())
            plt.title(f"Image {i}")
            plt.axis('off')
            plt.show()
        except:
            print(f"Failed to decode potential image at position {start_pos}")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install waymo-open-dataset-tf-2-14 -q
